In [ ]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape
from tqdm import tqdm


def tif_to_vector_by_blocks(
    tif_path, out_folder, out_file="vector.gpkg", layer_name=None
):
    if not os.path.exists(tif_path):
        raise FileNotFoundError(f"Raster file not found: {tif_path}")

    with rasterio.open(tif_path) as src:
        meta = src.meta.copy()
        crs = src.crs
        transform = src.transform
        nodata = src.nodata

        shapes_list = []

        # Recorremos por bloques
        for ji, window in tqdm(src.block_windows(1), desc="Procesando bloques"):
            data = src.read(1, window=window)
            mask = data > 0

            if np.any(mask):
                window_transform = src.window_transform(window)
                for g, val in shapes(data, mask=mask, transform=window_transform):
                    shapes_list.append(
                        {"geometry": shape(g), "properties": {"value": int(val)}}
                    )

    print(f"Total shapes found: {len(shapes_list)}")
    gdf = gpd.GeoDataFrame.from_features(shapes_list, crs=crs)

    os.makedirs(out_folder, exist_ok=True)
    out_path = os.path.join(out_folder, out_file)
    ext = os.path.splitext(out_file)[1].lower()

    print(f"Saving to {out_path} with extension {ext}")
    if ext == ".gpkg":
        if layer_name is None:
            layer_name = os.path.splitext(out_file)[0]
        gdf.to_file(out_path, driver="GPKG", layer=layer_name)
    elif ext == ".shp":
        gdf.to_file(out_path, driver="ESRI Shapefile")
    elif ext in [".geojson", ".json"]:
        gdf.to_file(out_path, driver="GeoJSON")
    else:
        raise ValueError(f"Unsupported output format: {ext}")

    return out_path

In [ ]:
tif_to_vector_by_blocks(
    "../results/adef_intg_forest_masked.tif",
    "../results/",
    out_file="adef_intg_forest_masked_by_blocks.gpkg",
    layer_name="adef_intg_forest_masked_by_blocks",
)

In [ ]:
os.system()

In [ ]:
import os
from osgeo import gdal, ogr, osr


def tif_to_vector_gdal(tif, out_folder, out_file="vector.gpkg", layer_name=None):
    """
    Converts a raster file (TIF) to a vector file using the GDAL Python bindings.

    Args:
        tif (str): Path to the input raster file.
        out_folder (str): Output folder for the vector file.
        out_file (str, optional): Output filename (e.g., 'vector.gpkg'). Defaults to 'vector.gpkg'.
        layer_name (str, optional): Name of the output layer. Defaults to file name without extension.

    Raises:
        FileNotFoundError: If the input TIF does not exist.
    """
    if not os.path.exists(tif):
        raise FileNotFoundError(f"Input file not found: {tif}")

    if not os.path.exists(out_folder):
        os.makedirs(out_folder)

    out_vector = os.path.join(out_folder, out_file)
    driver_name = _get_gdal_driver(out_file)
    if layer_name is None:
        layer_name = os.path.splitext(os.path.basename(out_file))[0]

    print(f"Opening raster: {tif}")
    src_ds = gdal.Open(tif)
    band = src_ds.GetRasterBand(1)

    # Create output data source
    drv = ogr.GetDriverByName(driver_name)
    if os.path.exists(out_vector):
        drv.DeleteDataSource(out_vector)

    dst_ds = drv.CreateDataSource(out_vector)
    srs = osr.SpatialReference()
    srs.ImportFromWkt(src_ds.GetProjection())

    out_layer = dst_ds.CreateLayer(layer_name, srs=srs, geom_type=ogr.wkbPolygon)
    field_defn = ogr.FieldDefn("value", ogr.OFTInteger)
    out_layer.CreateField(field_defn)

    print("Running GDAL Polygonize...")
    gdal.Polygonize(band, None, out_layer, 0, [], callback=None)

    # Cleanup
    dst_ds.Destroy()
    src_ds = None

    print(f"Done. Vector saved to: {out_vector}")


def _get_gdal_driver(filename):
    ext = filename.lower().split(".")[-1]
    if ext == "gpkg":
        return "GPKG"
    elif ext in ["shp"]:
        return "ESRI Shapefile"
    elif ext in ["json", "geojson"]:
        return "GeoJSON"
    else:
        raise ValueError(f"Unsupported file format: {ext}")

In [ ]:
tif_to_vector_gdal(
    "../results/adef_intg_forest_masked.tif",
    "../results/",
    out_file="adef_intg_forest_masked_by_gdal.gpkg",
    layer_name="adef_intg_forest_masked_by_gdal",
)

In [ ]:
import geopandas as gpd

# Crear a gdf a partir de los shapes
geoms = gpd.GeoDataFrame.from_records(
    ((shape(geom), value) for geom, value in shapes),
    columns=["geometry", "value"],
)

In [ ]:
False + 1

In [ ]:
# Importar el modulo editable
from adef_intg import utils_adef
from adef_intg import locking
from pathlib import Path
import rioxarray as rxr
import threading

In [ ]:
lock_read = locking.get_safe_lock("rio-read")
lock_read

In [ ]:
lock_write = locking.get_safe_lock("rio-write")
lock_write

In [ ]:
vector = utils_adef.get_wfs_layer(
    wfs_url="https://geoserver.icf.gob.hn/icfpub/wfs",
    layer_name="icfpub:limite_departamentos_gp",
    version="1.1.0",
)

In [ ]:
url_adef_intg = (
    "https://data-api.globalforestwatch.org/dataset/gfw_integrated_alerts/"
    "latest/download/geotiff?grid=10/100000&tile_id=20N_090W&pixel_meaning="
    "date_conf&x-api-key=2d60cd88-8348-4c0f-a6d5-bd9adb585a8c"
)
tif = rxr.open_rasterio(
    url_adef_intg,
    chunks=True,
    # lock=False,
    lock=threading.Lock(),
)

In [ ]:
utils_adef.clip_tif_to_ext(tif=tif, vector=vector, out_tif="../data/adef_intg_hn.tif")

In [ ]:
base_dir = Path("../").resolve()
confidence = 1
utils_adef.mask_adef_hn_by_forest(
    tif_forest18=base_dir / "data/bosque18_lzw.tif",
    tif_forest24=base_dir / "data/bosque24_lzw.tif",
    tif_forest14=base_dir / "data/bosque14_lzw.tif",
    tif_adef_hn=base_dir / "data/adef_intg_hn.tif",
    tif_forest14_match=base_dir / "data/bosque14_lzw_match.tif",
    tif_forest18_match=base_dir / "data/bosque18_lzw_match.tif",
    tif_forest24_match=base_dir / "data/bosque24_lzw_match.tif",
    tif_out=base_dir / "results/adef_intg_hn_forest_masked.tif",
    confidence_integ=confidence,
)

# Configuraciones y librerias

In [ ]:
# Librerias
import os
import sys
import time
import warnings
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import threading
from pathlib import Path
from owslib.wfs import WebFeatureService
from shapely.errors import ShapelyDeprecationWarning

# Ignorar advertencias de GeoPandas relacionadas con CRS
warnings.filterwarnings("ignore", message="Geometry is in a geographic CRS.")

# Opcional: Ignorar advertencias de Shapely si aparecen
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

# Agregar la ruta del directorio padre al sys.path
try:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
    base_dir = Path(__file__).resolve().parent.parent
except:
    __file__ = os.path.abspath("adef_intg.ipynb")
    sys.path.append(os.path.dirname(os.path.dirname(__file__)))
    base_dir = Path(__file__).resolve().parent.parent

from src import utils_adef


def run_adef_process(
    confidence, out_folder, out_file, layer_name, start_date, end_date, base_dir
):
    start_time = time.time()
    print(
        f"🚀 Iniciando el procesamiento de alertas integradas a las: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))} 🌍"
    )
    # Definir la URL de las alertas
    url_adef_intg = (
        "https://data-api.globalforestwatch.org/dataset/gfw_integrated_alerts/"
        "latest/download/geotiff?grid=10/100000&tile_id=20N_090W&pixel_meaning="
        "date_conf&x-api-key=2d60cd88-8348-4c0f-a6d5-bd9adb585a8c"
    )

    # Descargar los tif de bosque
    TIFS = {
        "bosque14_lzw": "https://git.icf.gob.hn/alopez/adef-integ-tools/-/raw/main/data/bosque14_lzw.tif",
        "bosque18_lzw": "https://git.icf.gob.hn/alopez/adef-integ-tools/-/raw/main/data/bosque18_lzw.tif",
        "bosque24_lzw": "https://git.icf.gob.hn/alopez/adef-integ-tools/-/raw/main/data/bosque24_lzw.tif",
    }
    for tif_name, tif_url in TIFS.items():
        tif_path = base_dir / "data" / f"{tif_name}.tif"
        if not tif_path.exists():
            print(f"Descargando {tif_name}...")
            utils_adef.dw_tif(
                url=tif_url,
                tif_out=tif_path,
            )
            print(f"{tif_name} descargado y guardado en {tif_path}")
        else:
            print(f"{tif_name} ya existe en {tif_path}, se omitirá la descarga.")

    # Preparar la data para análisis
    ## Preparar los datos auxiliares
    # Crear la conexion al servicio WFS
    url_icf_wfs = "https://geoserver.icf.gob.hn/icfpub/wfs"
    wfs_icf = WebFeatureService(url_icf_wfs, version="1.1.0")

    # Obtener el GeoDataFrame de los departamentos de Honduras
    lyr_dep = "icfpub:limite_departamentos_gp"
    dep_response = wfs_icf.getfeature(typename=lyr_dep, outputFormat="application/json")
    gdf_dep = gpd.read_file(dep_response)

    # Preparar el tif por el área y fechas de interés
    print("...Iniciando el enmascaramientos de las alertas integradas")
    # Leer el tif de alertas
    tif_adef_intg = rxr.open_rasterio(url_adef_intg, lock=True, chunks=True)

    # Cortar por el extend de Honduras
    if gdf_dep.crs != tif_adef_intg.rio.crs:
        gdf_dep_reproyectado = gdf_dep.to_crs(tif_adef_intg.rio.crs)
        tif_adef_intg_hn = tif_adef_intg.rio.clip_box(
            *gdf_dep_reproyectado.total_bounds
        )
        tif_adef_intg_hn.name = "adef_intg_hn"
        print(
            "Se realizó clip al raster con el GeoDataFrame de departamentos reproyectado"
        )
    else:
        tif_adef_intg_hn = tif_adef_intg.rio.clip_box(*gdf_dep.total_bounds)
        tif_adef_intg_hn.name = "adef_intg_hn"

    if start_date and end_date:
        tif_adef_intg_hn, _, _ = utils_adef.filter_adef_intg_time(
            tif_adef_intg_hn,
            ("Range", start_date, end_date),
            base_dir / "data/adef_intg_hn.tif",
        )
        print(f"Se realizó el filtrado por fechas {start_date} - {end_date}")
    else:
        tif_adef_intg_hn.rio.to_raster(
            base_dir / "data/adef_intg_hn.tif",
            tiled=True,
            lock=threading.Lock(),
            compress="DEFLATE",
        )
        print("Se procesó el raster sin filtrar por fechas")

    utils_adef.mask_adef_hn_by_forest(
        tif_forest18=base_dir / "data/bosque18_lzw.tif",
        tif_forest24=base_dir / "data/bosque24_lzw.tif",
        tif_forest14=base_dir / "data/bosque14_lzw.tif",
        tif_adef_hn=base_dir / "data/adef_intg_hn.tif",
        tif_forest14_match=base_dir / "data/bosque14_lzw_match.tif",
        tif_forest18_match=base_dir / "data/bosque18_lzw_match.tif",
        tif_forest24_match=base_dir / "data/bosque24_lzw_match.tif",
        tif_out=base_dir / "results/adef_intg_hn_forest_masked.tif",
        confidence_integ=confidence,
    )
    print("Se realizó el enmascaramiento de las alertas integradas por el bosque")

    # Crear el vector de alertas
    print("...creando el vector de las alertas integradas")

    # Crear gpkg de las alertas
    utils_adef.tif_to_vector(
        tif=base_dir / "results/adef_intg_hn_forest_masked.tif",
        out_folder=out_folder,
        out_file=out_file,
        layer_name=layer_name,
    )
    # Agregar la fecha de la alerta y actualizar los datos de la capa
    gdf = gpd.read_file(os.path.join(out_folder, out_file), layer=layer_name)
    gdf = utils_adef.calculate_decompose_date(gdf, "value", "INTEGRATED")
    gdf["confidence"] = gdf["value"] // 10000
    gdf.to_file(
        out_folder / out_file,
        layer=layer_name,
        driver="GPKG",
        index=False,
    )
    time_end = time.time()
    print(
        f"Finalizando el procesamiento de alertas integradas a las: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time_end))}"
    )
    elapsed_time = time_end - start_time
    hours, remainder = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(
        f"El tiempo de procesamiento fue de: {int(hours)} horas, {int(minutes)} minutos y {seconds:.2f} segundos"
    )
    return gdf